# Prediccion y Clasificación de Alquileres en Madrid

-  Proyecto orientado al analisis, prediccion y clasificacion de alquileres en la provincia de Madrid mediante tecnicas de ciencia de datos y machine learning.

Objetivos principales:
- Predecir el precio de alquiler de una vivienda (tarea de regresion).
- Predecir si un inmueble dispone de balcon (tarea de clasificacion).

Índice del notebook

## 1. Inicio y Configuración

#### Importacion de librerias

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# TRAIN  -TEST - METRICAS
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import (accuracy_score, f1_score, precision_score, recall_score,  confusion_matrix, classification_report,balanced_accuracy_score,r2_score,mean_absolute_error, mean_squared_error
)

In [ ]:
# PREPROCESAMIENTO 
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer, make_column_selector,make_column_transformer
from sklearn.preprocessing import(MinMaxScaler, OneHotEncoder, StandardScaler,RobustScaler,PowerTransformer,QuantileTransformer)
from sklearn.impute import SimpleImputer,KNNImputer
from sklearn.pipeline import FunctionTransformer
from sklearn import clone

In [ ]:
# MODELOS CLASIFICACION
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# MODELO REGRESION
from sklearn.linear_model import LinearRegression

# VALIDACION CRUZADA
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.model_selection import cross_validate


### Carga del dataset

In [2]:
df = pd.read_csv('idealista-madrid.csv')
df.sample(n=3)

,web_id,url,title,type,price,deposit,private_owner,professional_name,floor_built,floor_area,...,terrace,balcony,storeroom,swimming_pool,garden_area,location,district,subdistrict,postalcode,last_update
5921,99079842,https://www.idealista.com/en/inmueble/99079842/,Flat / apartment for rent in calle de Juan Per...,Flat,1100,NaN,False,Housfy Alquiler,95,79.0,...,False,True,False,False,False,"Calle de Juan Perez Almeida, Subdistrict Comil...",Carabanchel,Comillas,28019.0,31 October
861,39861421,https://www.idealista.com/en/inmueble/39861421/,Flat / apartment for rent in Las Tablas,Flat,900,NaN,False,Promora Las Tablas,54,NaN,...,False,False,True,False,False,", Subdistrict Las Tablas, District Fuencarral,...",Fuencarral,Las Tablas,28034.0,7 November
8204,97416760,https://www.idealista.com/en/inmueble/97416760/,Flat / apartment for rent in calle de Sagasta,Flat,3350,NaN,False,Js gestion inmobiliaria,120,110.0,...,True,False,False,False,False,"Calle de Sagasta, Subdistrict Chueca-Justicia,...",Centro,Chueca-Justicia,28004.0,21 April


## 2. Entendimiento de los Datos

### Inspeccion del DataSet

In [3]:
df.head(1)

,web_id,url,title,type,price,deposit,private_owner,professional_name,floor_built,floor_area,...,terrace,balcony,storeroom,swimming_pool,garden_area,location,district,subdistrict,postalcode,last_update
0,99439319,https://www.idealista.com/en/inmueble/99439319/,"Flat / apartment for rent in pablo luna, 4",Flat,1400,NaN,False,Silcasas Ochocientas,60,NaN,...,True,False,False,False,False,"pablo luna, 4, Subdistrict Castilla, District ...",Chamartín,Castilla,28046.0,7 November


- tipos de datos

In [4]:
print(
    f"DATA FRAME\n"
    f"Numero de filas y columnas:{df.shape}\n"
    f"Numero total de elementos : {df.size}\n "
    )

print(df.dtypes.to_frame(name="TIPOS DE DATOS"))

DATA FRAME
Numero de filas y columnas:(9229, 32)
Numero total de elementos : 295328
 
                  TIPOS DE DATOS
web_id                     int64
url                          str
title                        str
type                         str
price                      int64
deposit                  float64
private_owner               bool
professional_name            str
floor_built                int64
floor_area               float64
floor                        str
year_built               float64
orientation                  str
bedrooms                   int64
bathrooms                  int64
second_hand                 bool
lift                        bool
garage_included             bool
furnished                   bool
equipped_kitchen            bool
fitted_wardrobes            bool
air_conditioning            bool
terrace                     bool
balcony                     bool
storeroom                   bool
swimming_pool               bool
garden_area            

### Redimensionamiento dataset

- Eliminamos columnas no utiles para el proyecto.

In [5]:
# hacemos una copia del df original y realizamos el drop de la columna que no queremos y mostramos el resultado
df_copy = df.copy()

cols_a_eliminar = ["location", "web_id", "url", "subdistrict"] 
df_new = df_copy.drop(columns=cols_a_eliminar)

df_new.columns.to_list()


['title',
 'type',
 'price',
 'deposit',
 'private_owner',
 'professional_name',
 'floor_built',
 'floor_area',
 'floor',
 'year_built',
 'orientation',
 'bedrooms',
 'bathrooms',
 'second_hand',
 'lift',
 'garage_included',
 'furnished',
 'equipped_kitchen',
 'fitted_wardrobes',
 'air_conditioning',
 'terrace',
 'balcony',
 'storeroom',
 'swimming_pool',
 'garden_area',
 'district',
 'postalcode',
 'last_update']

In [6]:
# solo dtypes numéricos
df_new.select_dtypes(include='number').dtypes

price            int64
deposit        float64
floor_built      int64
floor_area     float64
year_built     float64
bedrooms         int64
bathrooms        int64
postalcode     float64
dtype: object

### Revision NULOS - VALORES OCULTOS -  ESPACIOS EN BLANCOS - VALORES = 0
  - comprobamos si hay VALORES NULOS  en todo el dataframe reduced
  - comprobamos si hay ESPACIOS EN BLANCO en todo el dataframe reduced
  - comprobamos si hay VALORES OCULTOS  en todo el dataframe reduced
  - comprobamos si hay VALORES IGUAL A ZEROS en todo el dataframe reduced

In [7]:
df_nulos = pd.DataFrame({    
    'nulos': df_new.isna().sum()                                                        # muestra la suma de los nulos o faltantes contenidos en las columnas 
})

df_espacios = pd.DataFrame({
    "espacios": df_new.apply(lambda col: col.astype(str).str.strip().eq("").sum())      # muestra la suma de celdas con espacios vacios
})

valores_ocultos = ["", " ", "NA", "N/A", "na", "null", "None", "?", ".", "0"]

df_ocultos = pd.DataFrame({ 
    'ocultos': df_new.apply(lambda col: col.astype(str).isin(valores_ocultos).sum()) })  #  trasforma todas las columnas en string para poder comparar con valores_ocultos 

df_zeros= pd.DataFrame({
    "valores  = 0 ":df_new.apply( lambda col: (col == 0).sum())                           # suma todos los valores iguales a zeros de todas las columnas
})

df_Nan= pd.concat([df_nulos, df_espacios, df_ocultos, df_zeros], axis=1)
df_Nan

,nulos,espacios,ocultos,valores = 0
title,0,0,0,0
type,0,0,0,0
price,0,0,0,0
deposit,3822,0,0,0
private_owner,0,0,0,7622
professional_name,1607,0,0,0
floor_built,0,0,9,9
floor_area,5291,0,0,1
floor,321,0,0,0
year_built,6336,0,0,0


- El DataFrame muestra que el DataSet presenta variedad de valores respecto a: nulos , valor a zero, y oculto. 
  Hay feature que presentan valores a 0 que interpretamos que es debido a que no existe esta caracteristica en los piso ( ejemplo: garden_area) y habar que tenerlo presente en nuestro estudio y valorar como tratar estos datos.

- mostramos las filas cuyos valores sean indenticos en sus columnas.

In [8]:
num_cols = df_new.select_dtypes(include='number').columns.tolist()  # seleccion de columnas numericas.

df_new.groupby(num_cols).size().reset_index(name='count').query('count > 1').sort_values('count', ascending=False) # agrupamos los valores repetidos y los ordenamos.

,price,deposit,floor_built,floor_area,year_built,bedrooms,bathrooms,postalcode,count
86,850,1.0,56,52.0,1961.0,2,1,28042.0,2
337,1350,2.0,95,80.0,1962.0,2,1,28008.0,2
448,1750,1.0,159,129.0,2004.0,4,3,28222.0,2
521,2230,1.0,57,56.0,1903.0,1,1,28010.0,2
538,2300,3.0,136,118.0,1971.0,4,3,28046.0,2


- Resultado: tenemos 5 grupos con un total de 10 filas repetidas. El modelo no se verá afectado por estas cantidades.

In [9]:
# solo dtypes string
df_new.select_dtypes(include='str').dtypes

title                str
type                 str
professional_name    str
floor                str
orientation          str
district             str
last_update          str
dtype: object

## 3. EDA

### VALORES NUMERICOS
Tabla de estadisticos principal para ver centralidad y dispercion

Renombramos las descripciones, para practicar realizar el diccionario y para que se lea mas rapido en español.

In [10]:
# describir solo las numéricas
df_new.describe(include='number').round(3)

,price,deposit,floor_built,floor_area,year_built,bedrooms,bathrooms,postalcode
count,9229.000,5407.000,9229.000,3938.000,2893.000,9229.000,9229.000,6834.000
mean,1937.996,1.471,110.285,94.357,1975.492,2.260,1.788,28076.948
std,1615.063,0.622,87.184,72.823,34.929,1.333,1.077,171.686
min,400.000,1.000,0.000,0.000,1800.000,0.000,1.000,28001.000
25%,1000.000,1.000,60.000,54.000,1960.000,1.000,1.000,28009.000
50%,1400.000,1.000,85.000,75.000,1978.000,2.000,2.000,28023.000
75%,2300.000,2.000,127.000,110.000,2003.000,3.000,2.000,28043.000
max,25000.000,6.000,990.000,995.000,2022.000,25.000,20.000,28950.000


 REPORT columnas numericas:
- en 'price' existe una diferencia significativa entre media y mediana, confirmandono presencia de valores outliers. El rango entre valores min y max es dispar. La desviacion estandar tiene un valor muy elevado respecto a la media( aprox. 83%) que tendremos que tener en cuenta y tratar.
- en 'deposit' faltan muchos registros y presenta una desviacion elevada. No es un dato util para el modelo y es cuestionable si utilizarlo o no. 
- en 'floor_built' presentan valores minimos a 0. Como revisamos anteriormente, hemos comprobado que solo afecta a 9 registros, que resultan de escasa importancia.Solo abria que comprobar que este 0 sea un por ejemplo un Bajo.  Media y mediana estan relativamente cerca y no hay mucha desviacion.
- en 'floor area' faltan muchos registros, y esto puede ser un problema por ser factor de peso para la predicción del  precio y la variable 'balcony'.
- 'floor_built" e "floor_area" tienen valores muy parecido, pero en 'area' faltan muchos datos. Podriamos utilizar una valoracion cruzada, utilizando solo 'floor_build" para evitar introducir datos innecesarios al modelo.
- en 'years' a destacar la falta elevada de registros y podria ser un dato importante para el estudio. Una posible solucion seria la discretizacion en categorias ("pre_guerra" ,"Antiguos", "Moderno" , " Desconocido" )
- en ' bethroom' y 'bathroom' hay valores maximos muy altos, Habria que revisar de que se trata, probablemente son errores. Estos valroes sesgaria mucho el modelo. Habria que corregirlos o eliminarlos.
- en ' postalcode'  a destacar que faltan datos y podriamos realizar tal vez una extrapolacion de otra feature de tipo categorica que tenemos disponible(' location') que en caso fuera necesario podriamos realizar la operacion.

CONCLUSION: 
Es importante tener en cuenta la lata dispercion en el precio ( un 83% de la media).
Revisar los valores maximo de bethrooms y bathrooms.
La falta de datos en 'years' es critica , siendo importante para una predicción en el precio.
Considerar la idea de utilizar solo floor_built para evitar introducir demasiados datos al modelo que no sean necesarios. 

## ANALISIS 


### VALORES CATEGORICOS

In [ ]:
# Balanceo

# ANÁLISIS DE BALANCEO PARA 'BALCONY' (Clasificación)
df_balanceo_balcony = pd.DataFrame({
    "conteo": df_new['balcony'].value_counts(),
    "porcentaje": (df_new['balcony'].value_counts(normalize=True) * 100).round(2)
})

sns.countplot(data=df_new, x='balcony')
plt.title('Distribución de Balcony')
plt.show()

df_balanceo_balcony


In [ ]:
#CODIGO 

INTERPRETACION RESULTADOS 

## 4. Preparación de Datos - PREPROCESSING

Seleccion incial de columnas y target

* Numericas: 'floor_built', 'floor_area', 'bathrooms', 'bedrooms', 'year_built'
* Categoricas: 'type', 'district'
* Target:
   * Regresion: 'priece' 
   * Clasificacion binaria: 'balcony'

#### Para REGRESION

In [ ]:
df_reg = df.dropna(subset=['price']).copy()
X_rg= df_reg[["floor_built", "floor_area", "bathrooms" ,"bedrooms", "year_built", "type","district"]]
y_rg= df_reg['price']

X_rg_train, X_rg_test, y_rg_train, y_rg_test = train_test_split(X_rg, y_rg, test_size=0.20, random_state=42)
print(f"{'Train:', X_rg_train.shape}")
print(f"{'Test:', X_rg_test.shape}")

('Train:', (7383, 7))
('Test:', (1846, 7))


- PIPELINE

In [ ]:
numerical_cols_selector = make_column_selector (dtype_include=np.number)
categorical_cols_selector= make_column_selector(dtype_include=object)

numerical_pipeline= make_pipeline(
    SimpleImputer(strategy="median"),
    StandardScaler()
)

categorical_pipeline= make_pipeline(
    SimpleImputer(strategy="most_frequent"),
    OneHotEncoder(handle_unknown="ignore", sparse_output=False)
)

preprocesor= make_column_transformer(
    (numerical_pipeline, numerical_cols_selector),
    (categorical_pipeline, categorical_cols_selector)
)

preprocesor



,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('pipeline-1', ...), ('pipeline-2', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers contains sparse matrices,these will be stacked as a sparse matrix if the overall density islower than this value. Use ``sparse_threshold=0`` to always returndense. When the transformed output consists of all dense data, thestacked result will be dense, and this keyword will be ignored.",0.3
,"n_jobs n_jobs: int, default=NoneNumber of jobs to run in parallel.``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.``-1`` means using all processors. See :term:`Glossary `for more details.",None
,"transformer_weights transformer_weights: dict, default=NoneMultiplicative weights for features per transformer. The output of thetransformer is multiplied by these weights. Keys are transformer names,values the weights.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each transformer will beprinted as it is completed.",False
,"verbose_feature_names_out verbose_feature_names_out: bool, str or Callable[[str, str], str], default=True- If True, :meth:`ColumnTransformer.get_feature_names_out` will prefix all feature names with the name of the transformer that generated that feature. It is equivalent to setting `verbose_feature_names_out=""{transformer_name}__{feature_name}""`.- If False, :meth:`ColumnTransformer.get_feature_names_out` will not prefix any feature names and will error if feature names are not unique.- If ``Callable[[str, str], str]``, :meth:`ColumnTransformer.get_feature_names_out` will rename all the features using the name of the transformer. The first argument of the callable is the transformer name and the second argument is the feature name. The returned string will be the new feature name.- If ``str``, it must be a string ready for formatting. The given string will be formatted using two field names: ``transformer_name`` and ``feature_

In [ ]:
pipeline_regression = make_pipeline(preprocesor,LinearRegression())
pipeline_regression.fit(X_rg_train, y_rg_train)
pipeline_regression

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('linearregression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('pipeline-1', ...), ('pipeline-2', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of

In [ ]:
y_pred = pipeline_regression.predict(X_rg_test)

metrics_df = pd.DataFrame({
    "Metric": ["R2", "MAE", "MSE", "RMSE"],
    "Value": [
        r2_score(y_rg_test, y_pred),
        mean_absolute_error(y_rg_test, y_pred),
        mean_squared_error(y_rg_test, y_pred),
        np.sqrt(mean_squared_error(y_rg_test, y_pred))
    ]
})

metrics_df


,Metric,Value
0,R2,0.647728
1,MAE,536.543675
2,MSE,784448.899847
3,RMSE,885.691199


In [ ]:
regresion_precios = pd.DataFrame({'price_real': y_rg_test.values, 'price_pred':y_pred})
regresion_precios.head(5)

,price_real,price_pred
0,2300,2482.685689
1,4200,3889.705046
2,1500,1754.076693
3,2200,1458.826152
4,800,1314.821276


por cada precio real, se indica lo que hubiera prdecido el modelo.

#### Para CLASIFICACION

- clonamos el Preprocesado realizado con REGRESION

In [73]:
preprocesor_clasification = clone (preprocesor)
pipeline_clasificacion = make_pipeline(
    preprocesor_clasification,
    LogisticRegression(max_iter=500)
)
pipeline_clasificacion

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('logisticregression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('pipeline-1', ...), ('pipeline-2', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output 

In [ ]:
df_clas = df.dropna(subset=['balcony']).copy()
X_cl= df_clas[["floor_built", "floor_area", "bathrooms" ,"bedrooms", "year_built", "type","district"]]  # selecion de las mismas feature utilizadas en REGRESION
y_cl= df_clas['balcony']

X_cl_train, X_cl_test, y_cl_train, y_cl_test = train_test_split(X_cl, y_cl, test_size=0.20, random_state=42)
print(f"{'Train:', X_cl_train.shape}")
print(f"{'Test:', X_cl_test.shape}")


('Train:', (7383, 7))
('Test:', (1846, 7))


In [79]:
pipeline_clasificacion.fit(X_cl_train, y_cl_train)
pipeline_clasificacion

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('columntransformer', ...), ('logisticregression', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('pipeline-1', ...), ('pipeline-2', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output 

In [83]:
y_cl_pred = pipeline_clasificacion.predict(X_cl_test)

metrics_df = pd.DataFrame({
    "Metric": ["ACCURACY", "F1", "BALANCED", "PRECISION", "RECALL"],
    "Value": [
        accuracy_score(y_cl_test, y_cl_pred),
        f1_score(y_cl_test, y_cl_pred),
        balanced_accuracy_score(y_cl_test, y_cl_pred),
        precision_score(y_cl_test, y_cl_pred),
        recall_score(y_cl_test, y_cl_pred)
    ]
})

metrics_df

,Metric,Value
0,ACCURACY,0.822860
1,F1,0.046647
2,BALANCED,0.507016
3,PRECISION,0.320000
4,RECALL,0.025157


## 5. Modelado

#### Para REGRESION

#### Para CLASIFICACION

## 6. Conclusiones